## Optimtize To get over 75%!

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

from tensorflow.keras.models import load_model
model = load_model("AlphabetSoupCharity.h5")

application_df.head()

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
application_types_to_replace = ['T9','T2','T13','T12','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_value_counts = application_df['CLASSIFICATION'].value_counts().index.tolist()

classifications_to_replace = classification_value_counts[5:]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
application_dummies_df = pd.get_dummies(application_df)

application_dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df['IS_SUCCESSFUL']
X = application_dummies_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## More epochs!

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model with 200 epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5787 - accuracy: 0.7149
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5575 - accuracy: 0.7261
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5537 - accuracy: 0.7294
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5514 - accuracy: 0.7296
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5508 - accuracy: 0.7317
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5487 - accuracy: 0.7326
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5484 - accuracy: 0.7318
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7327
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5466 - accuracy: 0.7331
Epoch 10/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5465 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5361 - accuracy: 0.7395
Epoch 82/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5356 - accuracy: 0.7397
Epoch 83/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5358 - accuracy: 0.7402
Epoch 84/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5348 - accuracy: 0.7400
Epoch 85/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5352 - accuracy: 0.7400
Epoch 86/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5355 - accuracy: 0.7409
Epoch 87/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5357 - accuracy: 0.7397
Epoch 88/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5354 - accuracy: 0.7400
Epoch 89/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7401
Epoch 90/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5352 - accuracy: 

804/804 [==============================] - 3s 4ms/step - loss: 0.5327 - accuracy: 0.7411
Epoch 161/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5330 - accuracy: 0.7418
Epoch 162/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5330 - accuracy: 0.7411
Epoch 163/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5335 - accuracy: 0.7406
Epoch 164/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7410
Epoch 165/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7411
Epoch 166/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5325 - accuracy: 0.7410
Epoch 167/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5327 - accuracy: 0.7406
Epoch 168/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7412
Epoch 169/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5325 - a

### More layers

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 5

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 5)                 155       
                                                                 
 dense_6 (Dense)             (None, 1)                 6         
                                                                 
Total params: 6111 (23.87 KB)
Trainable params: 6111 (23.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5725 - accuracy: 0.7147
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5544 - accuracy: 0.7256
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5513 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5500 - accuracy: 0.7310
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5492 - accuracy: 0.7309
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5470 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5466 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5463 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5456 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5357 - accuracy: 0.7395
Epoch 82/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5357 - accuracy: 0.7398
Epoch 83/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5359 - accuracy: 0.7398
Epoch 84/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5364 - accuracy: 0.7388
Epoch 85/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5361 - accuracy: 0.7386
Epoch 86/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5358 - accuracy: 0.7392
Epoch 87/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5362 - accuracy: 0.7388
Epoch 88/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5361 - accuracy: 0.7394
Epoch 89/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5360 - accuracy: 0.7386
Epoch 90/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5356 - accuracy: 

## Bucket the ASK_AMT column


In [15]:
application_df['ASK_AMT'].value_counts()[5:]

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [21]:
ask_amounts_to_replace = application_df['ASK_AMT'].value_counts().index.tolist()[1:]

# Replace in dataframe
for amt in ask_amounts_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,">5000")
    
# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
>5000     8901
Name: ASK_AMT, dtype: int64

In [22]:
application_dummies_df = pd.get_dummies(application_df)

application_dummies_df

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_>5000
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34295,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34296,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
34297,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [23]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df['IS_SUCCESSFUL']
X = application_dummies_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_test_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3600      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6061 (23.68 KB)
Trainable params: 6061 (23.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5728 - accuracy: 0.7197
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5544 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5522 - accuracy: 0.7285
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5492 - accuracy: 0.7294
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5486 - accuracy: 0.7305
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5479 - accuracy: 0.7330
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5464 - accuracy: 0.7320
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5463 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5449 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5449 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5337 - accuracy: 0.7399
Epoch 82/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5334 - accuracy: 0.7401
Epoch 83/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5332 - accuracy: 0.7404
Epoch 84/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5334 - accuracy: 0.7408
Epoch 85/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5328 - accuracy: 0.7404
Epoch 86/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5330 - accuracy: 0.7411
Epoch 87/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5336 - accuracy: 0.7401
Epoch 88/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5332 - accuracy: 0.7404
Epoch 89/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5331 - accuracy: 0.7409
Epoch 90/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5329 - accuracy: 